# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770692683272                   -0.53    9.0         
  2   -2.772142496602       -2.84       -1.30    1.0    105ms
  3   -2.772170212978       -4.56       -2.65    1.0    132ms
  4   -2.772170720381       -6.29       -4.01    2.0    128ms
  5   -2.772170722836       -8.61       -4.43    2.0    153ms
  6   -2.772170723011       -9.76       -5.62    1.0    111ms
  7   -2.772170723015      -11.40       -6.30    2.0    128ms
  8   -2.772170723015      -13.65       -6.89    1.0    128ms
  9   -2.772170723015      -13.77       -7.16    2.0    128ms
 10   -2.772170723015   +  -13.94       -7.68    1.0    133ms
 11   -2.772170723015      -13.99       -8.98    2.0    130ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770838210556                   -0.52    9.0         
  2   -2.7

1.773557997670782

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770687934905                   -0.53    8.0         
  2   -2.772052700174       -2.86       -1.30    1.0    108ms
  3   -2.772082839735       -4.52       -2.72    1.0    138ms
  4   -2.772083416853       -6.24       -4.12    2.0    124ms
  5   -2.772083417805       -9.02       -5.09    2.0    168ms
  6   -2.772083417810      -11.32       -6.02    1.0    113ms
  7   -2.772083417811      -12.42       -6.81    2.0    130ms
  8   -2.772083417811   +  -14.07       -7.57    1.0    132ms
  9   -2.772083417811      -13.94       -8.43    2.0    134ms

Polarizability via ForwardDiff:       1.7725349634943004
Polarizability via finite difference: 1.773557997670782
